In [6]:
import pandas as pd
import numpy as np

BASE_FEATURES = "../all_stations_events/all_stations_events.csv"
# BASE_FEATURES = "./all_stations_events.csv"
features = pd.read_csv(BASE_FEATURES)

## 1. Age

According to Santander Cycles Survey of Q2 2017/2018 their main target group depicts of people between the **age 16 and 54**.<br>
They group their customers in the following age groups:

![](img/target.png)

As this plot shows they have more male users than female users, why we give maless a higher weight than females.<br>
Moreover we should weight the several age groups differently. 

Population data of 2015-2019 was found at the [London Datastore](https://data.london.gov.uk/dataset/projections/).

After preparing the data in a first way we have the age groups ordered by London districts:


In [7]:
age = pd.read_csv("age.csv")
del age["Sum Male"]
age.head(10)

,District,Ages sum,16-24M,16-24F,25-34M,25-34F,35-44M,35-44F,45-54M,45-54F,55-64M,55-64F,>65M,>65F,Year
0,Camden,199263,15875,16521,26876,25634,19397,18384,14351,13960,10219,9862,12465,15719,2015
1,Camden,203057,16277,15983,27264,25554,20025,18558,15062,14466,10602,10203,12900,16163,2016
2,Camden,206309,16426,16255,27676,25880,20385,18586,15292,14733,10852,10484,13246,16494,2017
3,Camden,209229,16693,16564,28119,25886,20434,18603,15522,15033,11122,10786,13635,16832,2018
4,Camden,211902,16799,16948,28477,25915,20688,18411,15501,15356,11539,11096,13962,17210,2019
5,City of London,8146,287,339,950,553,454,374,714,462,2405,382,618,608,2015
6,City of London,8427,334,384,967,576,481,346,724,465,2506,373,641,630,2016
7,City of London,8514,332,359,974,592,495,359,746,448,2547,376,635,651,2017
8,City of London,8870,363,352,1021,655,534,400,744,434,2662,389,650,666,2018
9,City of London,9177,375,367,1073,702,567,424,732,434,2747,413,665,678,2019


In [8]:
# Calculating percentage population augmentation from 2015 to 2019

dico = {}
for index, row in age.iterrows():
    array = dico.get(row["District"],[])
    array.append(row["Ages sum"])
    dico[row["District"]] = array

for district,array in dico.items():
    print(district, "\t\t", int(100*(array[-1] - array[1])/array[-1]))

Camden 		 4
City of London 		 8
Hackney 		 4
Hammersmith and Fulham 		 3
Haringey 		 2
Islington 		 3
Kensington and Chelsea 		 2
Lambeth 		 3
Lewisham 		 3
Newham 		 5
Southwark 		 4
Tower Hamlets 		 5
Wandsworth 		 3
Westminster 		 4


In [9]:
# Get coordinate of each district:

from NominatimLibrary import Locator
locator = Locator()

district_coords = {}

def locate(locator, district):
    elm = district_coords.get(district)
    if elm == None:
        try:
            district_coords[district] = locator.get_coordinates("" + district + ", London, UK")
            return district_coords[district]
        except Exception as e:
            print("Could not locate: ", e)
    else:
        return elm

age["District"].map(lambda x: locate(locator, x))
    
for i,j in district_coords.items():
    print(i,j)

Camden (51.5423045, -0.1395604)
City of London (51.5156177, -0.0919983)
Hackney (51.5432402, -0.0493621)
Hammersmith and Fulham (51.4920377, -0.2236401)
Haringey (51.58792985, -0.10541010599099)
Islington (51.5384287, -0.0999051)
Kensington and Chelsea (51.4989948, -0.1991229)
Lambeth (51.5013012, -0.117287)
Lewisham (51.4624325, -0.0101331)
Newham (51.52999955, 0.0293179602938221)
Southwark (51.5029222, -0.103458)
Tower Hamlets (51.49595675, -0.011744492532098)
Wandsworth (51.4570271, -0.1932607)
Westminster (51.4973206, -0.137149)


In [10]:
stations = pd.read_csv("../raw/rental_stations_saved.csv")
stations.head()

,name,id,lat,lon,capacity
0,"River Street , Clerkenwell",1,51.529163,-0.109971,19
1,"Phillimore Gardens, Kensington",2,51.499607,-0.197574,37
2,"Christopher Street, Liverpool Street",3,51.521284,-0.084606,32
3,"St. Chad's Street, King's Cross",4,51.530059,-0.120974,23
4,"Sedding Street, Sloane Square",5,51.493130,-0.156876,27


In [11]:
# Get closest district for each station:

closest_col = []
for index, row in stations.iterrows():
    (lat,lon) = (row.lat,row.lon)
    closest_district = None
    closest_dist = 9999999
    for district,coords in district_coords.items():
        dist = locator.distance_crow_coords(coords, (lat,lon))
        if closest_district == None or dist < closest_dist:
            closest_dist = dist
            closest_district = district
    closest_col.append(closest_district)

stations["closest_district"] = closest_col
stations.to_csv("../raw/rental_stations_district.csv")
stations.head()

,name,id,lat,lon,capacity,closest_district
0,"River Street , Clerkenwell",1,51.529163,-0.109971,19,Islington
1,"Phillimore Gardens, Kensington",2,51.499607,-0.197574,37,Kensington and Chelsea
2,"Christopher Street, Liverpool Street",3,51.521284,-0.084606,32,City of London
3,"St. Chad's Street, King's Cross",4,51.530059,-0.120974,23,Islington
4,"Sedding Street, Sloane Square",5,51.493130,-0.156876,27,Westminster


In [12]:
features_d = features.merge(stations[["id","closest_district"]], how="left", left_on="Station ID", right_on="id")
features_d.head()

,Unnamed: 0,Station ID,Year,Month,Day of Month,Day of Year,Day of Week,Season,Holiday,Daily Weather,...,Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),Rented Bikes,Rented Bikes (Future),nearestEvent,nearestEvent (Future),nbCloseEvents,nbCloseEvents (Future),id,closest_district
0,0,1,2015,1,4,4,7,Winter,False,fog,...,36.295,NaN,9,15,9999.0,9999.0,0.0,0.0,1,Islington
1,1,1,2015,1,5,5,1,Winter,False,partly-cloudy-day,...,46.740,36.295,15,15,9999.0,9999.0,0.0,0.0,1,Islington
2,2,1,2015,1,6,6,2,Winter,False,partly-cloudy-day,...,42.150,46.740,15,19,9999.0,9999.0,0.0,0.0,1,Islington
3,3,1,2015,1,7,7,3,Winter,False,partly-cloudy-night,...,45.450,42.150,19,14,9999.0,9999.0,0.0,0.0,1,Islington
4,4,1,2015,1,8,8,4,Winter,False,rain,...,46.200,45.450,14,22,9999.0,9999.0,0.0,0.0,1,Islington


In [19]:
features_age = features_d.merge(age, how="left", left_on=["closest_district","Year"], right_on=["District","Year"])
del features_age["id"]
del features_age["closest_district"]
del features_age["Unnamed: 0"]
features_age.to_csv("./all_stations_evts_age.csv")
features_age.columns

Index(['Station ID', 'Year', 'Month', 'Day of Month', 'Day of Year',
       'Day of Week', 'Season', 'Holiday', 'Daily Weather',
       'Daily Weather (Past)', 'Humidity', 'Humidity (Past)', 'Windspeed',
       'Windspeed (Past)', 'Apparent Temperature (Avg)',
       'Apparent Temperature (Avg) (Past)', 'Rented Bikes',
       'Rented Bikes (Future)', 'nearestEvent', 'nearestEvent (Future)',
       'nbCloseEvents', 'nbCloseEvents (Future)', 'District', 'Ages sum',
       '16-24M', '16-24F', '25-34M', '25-34F', '35-44M', '35-44F', '45-54M',
       '45-54F', '55-64M', '55-64F', '>65M', '>65F'],
      dtype='object')

In [132]:
age_stations = age[age["Year"] == 2019].merge(stations[["id","closest_district"]], how="right", left_on="District", right_on="closest_district")
del age_stations["id"]
del age_stations["closest_district"]

d_nb = age_stations.groupby(["District"]).count()
d_nb[["Year"]]

,Year
District,
Camden,69
City of London,110
Hackney,53
Hammersmith and Fulham,32
Islington,46
Kensington and Chelsea,118
Lambeth,73
Newham,2
Southwark,45


In [70]:
d_rented = features_age[["Rented Bikes","District"]].groupby(["District"]).sum()
d_rented

,Rented Bikes
District,
Camden,592450
City of London,1202336
Hackney,140368
Hammersmith and Fulham,33821
Islington,332246
Kensington and Chelsea,731817
Lambeth,806646
Newham,3174
Southwark,258546


In [139]:
r_per_s = []
r_per_p = []
s_per_p = []

d_demographic = age_stations.groupby(["District"]).head(1)

for i in range(len(d_rented)):
    r_per_s.append(int(d_rented.iloc[i]["Rented Bikes"]/d_nb.iloc[i]["Ages sum"]))
    r_per_p.append(d_rented.iloc[i]["Rented Bikes"]/d_demographic.iloc[i]["Ages sum"])
    s_per_p.append(int(d_demographic.iloc[i]["Ages sum"]/d_nb.iloc[i]["Ages sum"]))

    
d_final = d_nb[[]]
d_final["Nb inhabitants"] = d_demographic["Ages sum"].tolist()
d_final["Nb stations"] = d_nb["Ages sum"].tolist()
d_final["Nb rented"] = d_rented["Rented Bikes"].tolist()
d_final["r / station"] = r_per_s
d_final["r / person"] = r_per_p
d_final["person / station"] = s_per_p

if True:
    for c in d_demographic.columns[2:-1]:
        if c == "District":
            next

        array = []
        for i in range(len(d_rented)):
            array.append(int(100*d_demographic.iloc[i][c]/d_demographic.iloc[i]["Ages sum"]))

        d_final["r/ %s" % c] = array

d_final.sort_values("Nb inhabitants")

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

,Nb inhabitants,Nb stations,Nb rented,r / station,r / person,person / station,r/ 16-24M,r/ 16-24F,r/ 25-34M,r/ 25-34F,r/ 35-44M,r/ 35-44F,r/ 45-54M,r/ 45-54F,r/ 55-64M,r/ 55-64F,r/ >65M,r/ >65F
District,,,,,,,,,,,,,,,,,,
City of London,9177,110,1202336,10930,131.016236,83,4,3,11,7,6,4,7,4,29,4,7,7
Kensington and Chelsea,134271,118,731817,6201,5.450298,1137,5,5,10,9,10,9,8,8,6,7,8,10
Hammersmith and Fulham,153076,32,33821,1056,0.220943,4783,6,6,13,13,10,9,7,7,5,5,5,7
Islington,202860,46,332246,7222,1.637809,4410,7,8,16,15,10,8,6,6,4,4,4,5
Camden,211902,69,592450,8586,2.795868,3071,7,7,13,12,9,8,7,7,5,5,6,8
Westminster,217558,134,990930,7395,4.554785,1623,6,6,13,11,11,9,8,7,5,5,6,7
Hackney,227081,53,140368,2648,0.618141,4284,6,6,15,16,12,10,6,7,4,4,4,5
Tower Hamlets,258943,39,85783,2199,0.331281,6639,8,8,17,16,12,9,6,5,4,3,3,4
Southwark,267121,45,258546,5745,0.967898,5936,7,7,14,14,10,9,7,7,5,5,4,5


> As there is much more rented bikes in the City where there is least people living,
> might be better to look at number of people **working** in areas rather than **living** there and **touristic frequentation**.
> Or more **age/usage** direct relationship data from the rain company.

## 3. Earnings
According to Santander Cycles Survey of Q2 2017/2018 their main target group come up with an anual income between **20 k and more than 70 k**.<br>
On the one hand they have **casual users** with an anual income between **20-40 k**<br>
On the other hand they have **members** which have an average anual income between **40 - 75+ k**

![](img/income.png)

<br><br>
Extracted earnings from 2015-2018 from [Office for National Statistics](https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2016provisionalresults).</br>


In [50]:
earnings = pd.read_csv('earnings.csv')
earnings

,District,2015,2016,2017,2018
0,Camden,42161.6,43165.2,42406.0,45084.0
1,City of London,60351.2,62696.4,63642.8,68151.2
2,Hackney,36550.8,38854.4,39249.6,39124.8
3,Hammersmith and Fulham,36831.6,38818.0,42692.0,43097.6
4,Haringey,34008.0,32656.0,34470.8,35058.4
5,Islington,45099.6,47762.0,48984.0,48656.4
6,Kensington and Chelsea,35952.8,34039.2,38303.2,39993.2
7,Lambeth,41574.0,39301.6,40211.6,41646.8
8,Lewisham,33649.2,34403.2,34637.2,33919.6
9,Newham,32541.6,34377.2,36878.4,36296.0


The most used stations according to **income** data could be in: 

* City of London (Members)
* Tower Hamlets (Members)
* Westminster (Members)
* Haringey (Casual Users)
* Lewisham (Casual Users)
* Newham (Casual Users) 


### To Do:
* add datetime to data(maybe date of each day for one year?)
* assign earnings data to stations

## 4. Political data
Extracted election results from 2016 from [London Datastore](https://data.london.gov.uk/elections/).</br>


**Conservatives**: 
* low-emission buses
* car and van to be zero-emission by 2050
* plant a million trees in towns and cities to improve air
* 25-year environment plan<br>
**Labour**: <br>
* Clean Air Act to deal with illegal air quality
* safeguard habitats and species in the blue belts of seas and oceans
* ban on fracking
* plant a million trees
* ensure that 60% of the UK’s energy comes from zero-carbon or renewable sources by 2030<br>
**Liberal Democrats**<br>
* charge on disposable coffee cups to reduce waste
* diesel scrappage scheme, and a ban on the sale of diesel cars and small vans in the UK by 2025
* extend ultra-low emission zones to 10 more towns and cities
* Zero Carbon Britain Act to set new targets to reduce net greenhouse gas emissions by 80% by 2040 and to zero by 2050<br>
**Green Party**<br>
* new Environmental Protection Act and a new environmental regulator and court
* end plastic waste by introducing a bottle deposit scheme
* a Clean Air Acta
* End the reliance on fossil fuels with a ban on fracking and pledge to bring forward the coal phase out by two years to 2023
* Scrap plans for all new nuclear power stations<br>
**UK Independence Party**
* Cancellation of any state financing of climate protection
***

-> Labour & Green Party have probably voters with a higher focus on environmental protection<br>
-> Liberal Democrats & Conservatives have probably voters who are amongst other things interested in environmental protection<br>
-> UK Independence Party have probably voters who do not care on environmental protection
<br><br>
-> Districts with **lots of** votes for **Green Party** & **Labour Party** and **less** votes for **UK Independence Party** probably have a higher use of rental bikes. 
<br><br>Reference: [The Guardian](https://www.theguardian.com/environment/2017/may/21/how-do-the-four-main-parties-compare-on-the-environment)

In [7]:
political = pd.read_csv('political.csv')
political

,District,% Con,% Lab,% Lib Dem,% Green,% UKIP
0,City of London,40.415105,37.647737,6.630153,7.004901,2.075526
1,Camden,27.284130,51.053337,5.129850,8.448074,2.033980
2,Hackney,12.093692,66.721942,2.896786,10.193162,1.382321
3,Hammersmith and Fulham,41.089339,39.768094,4.316411,5.916356,2.282932
4,Haringey,17.677652,60.070296,5.243635,8.783056,1.295669
5,Islington,16.990974,60.377152,4.623599,8.741961,2.371882
6,Kensington and Chelsea,55.599084,28.015772,4.075299,4.533198,1.836683
7,Lambeth,21.426381,56.114814,5.388028,9.172221,1.449483
8,Lewisham,19.395787,56.829223,4.828802,8.968610,2.947496
9,Newham,17.423339,65.307076,2.291541,4.473610,2.558457


In [10]:
# Use the stations districts from age section above

In [ ]:
BASE_FEATURES = "./all_stations_events.csv"
features = pd.read_csv(BASE_FEATURES)


The most used stations according to political data could be in: <br>
* Hackney
* Haringey
* Lambeth 
* Lewisham 
* Islington


### To Do:
* add datetime to data(maybe date of each day for the year?)
* assign political interests data to stations

# Adding extra features to base features dataset